Để giải quyết những từ hiếm (những từ ít xuất hiện trong văn bản), bên cạnh việc tách ký tự ta có thể sử dụng tách thành các subword. Vì cơ bản những từ hiếm vẫn được tạo thành từ những subword không hiếm. Byte Pair Encoding (PBE) là một trong những thuật toán hỗ trợ cho việc tách từ thành các subword được Philip Gage giới thiệu trong [A New Algorithm for Data Compression](https://www.drdobbs.com/a-new-algorithm-for-data-compression/184402829). Đây là một kỹ thuật nén dữ liệu hoạt động bằng cách thay thế các cặp byte liên tiếp có tần suất lớn bằng một byte không tồn tại trong dữ liệu. 

![image](https://i1.wp.com/trituenhantao.io/wp-content/uploads/2020/04/1.gif?resize=728%2C408&ssl=1)

# Train PBE

In [ ]:
import re
import pandas as pd
import numpy as np
from operator import itemgetter
from typing import Dict, Tuple, List, Set 

In [ ]:
vocab = {
    'l o w </w>': 5,
    'l o w e r </w>': 2,
    'n e w e s t </w>': 6,
    'w i d e s t </w>': 3,
    'h a p p i e r </w>': 2
}

In [ ]:
# Từ voacb ban đầu thực hiện đếm tần suất xuất hiện của các cặp ký tự liên tiếp
def get_pair_stats(voacb: Dict[str, int]) -> Dict[Tuple[str, str], int]:
  pairs = {}
  for word, freq in vocab.items():
    print("##################")
    print("word: {}, freq: {}".format(word, freq))
    symbols = word.split()
    print("symbols: ", symbols)
    for i in range(len(symbols) - 1):
      pair = (symbols[i], symbols[i+1])
      print("pair: ", pair)
      current_freq = pairs.get(pair, 0)
      print("current_freq: ", current_freq)
      pairs[pair] = current_freq + freq
      print("pairs[pair]: ", pairs[pair])
    print("################## \n")
  return pairs

In [ ]:
pair_stats = get_pair_stats(vocab)
pair_stats

##################
word: l o w </w>, freq: 5
symbols:  ['l', 'o', 'w', '</w>']
pair:  ('l', 'o')
current_freq:  0
pairs[pair]:  5
pair:  ('o', 'w')
current_freq:  0
pairs[pair]:  5
pair:  ('w', '</w>')
current_freq:  0
pairs[pair]:  5
################## 

##################
word: l o w e r </w>, freq: 2
symbols:  ['l', 'o', 'w', 'e', 'r', '</w>']
pair:  ('l', 'o')
current_freq:  5
pairs[pair]:  7
pair:  ('o', 'w')
current_freq:  5
pairs[pair]:  7
pair:  ('w', 'e')
current_freq:  0
pairs[pair]:  2
pair:  ('e', 'r')
current_freq:  0
pairs[pair]:  2
pair:  ('r', '</w>')
current_freq:  0
pairs[pair]:  2
################## 

##################
word: n e w e s t </w>, freq: 6
symbols:  ['n', 'e', 'w', 'e', 's', 't', '</w>']
pair:  ('n', 'e')
current_freq:  0
pairs[pair]:  6
pair:  ('e', 'w')
current_freq:  0
pairs[pair]:  6
pair:  ('w', 'e')
current_freq:  2
pairs[pair]:  8
pair:  ('e', 's')
current_freq:  0
pairs[pair]:  6
pair:  ('s', 't')
current_freq:  0
pairs[pair]:  6
pair:  ('t', '</w

{('a', 'p'): 2,
 ('d', 'e'): 3,
 ('e', 'r'): 4,
 ('e', 's'): 9,
 ('e', 'w'): 6,
 ('h', 'a'): 2,
 ('i', 'd'): 3,
 ('i', 'e'): 2,
 ('l', 'o'): 7,
 ('n', 'e'): 6,
 ('o', 'w'): 7,
 ('p', 'i'): 2,
 ('p', 'p'): 2,
 ('r', '</w>'): 4,
 ('s', 't'): 9,
 ('t', '</w>'): 9,
 ('w', '</w>'): 5,
 ('w', 'e'): 8,
 ('w', 'i'): 3}

In [ ]:
# Kết hợp những cặp ký tự thường gặp nhất 
def merge_vocab(best_pair: Tuple[str, str], vocab_in: Dict[str, int]) -> Dict[str, int]:
  vocab_out = {}
  # đảm bảo ký tự trong input pair là ký tự sẽ đc xử lý và ko coi nó là ký tự đặc biệt trong regular expression
  pattern = re.escape(' '.join(best_pair))
  print("pattern: ", pattern)
  replacement = ''.join(best_pair)
  print("replacement: ", replacement)
  for word_in in vocab_in:
    print("##################")
    print("word_in: ", word_in)
    word_out = re.sub(pattern, replacement, word_in)
    print("word_out: ", word_out)
    vocab_out[word_out] = vocab_in[word_in]
    print("################## \n")
  
  return vocab_out

In [ ]:
best_pair = max(pair_stats, key=pair_stats.get)
print("best_pair: ", best_pair)

new_vocab = merge_vocab(best_pair, vocab)
new_vocab

best_pair:  ('e', 's')
pattern:  e\ s
replacement:  es
##################
word_in:  l o w </w>
word_out:  l o w </w>
################## 

##################
word_in:  l o w e r </w>
word_out:  l o w e r </w>
################## 

##################
word_in:  n e w e s t </w>
word_out:  n e w es t </w>
################## 

##################
word_in:  w i d e s t </w>
word_out:  w i d es t </w>
################## 

##################
word_in:  h a p p i e r </w>
word_out:  h a p p i e r </w>
################## 



{'h a p p i e r </w>': 2,
 'l o w </w>': 5,
 'l o w e r </w>': 2,
 'n e w es t </w>': 6,
 'w i d es t </w>': 3}

phần code ở trên thể hiện cho 1 iter bây giờ sẽ tăng số iter lên và xem kết quả

In [ ]:
bpe_codes = {}
iter = 10
for i in range(iter):
  print("iter: ", i)
  pair_stats = get_pair_stats(vocab)
  if not pair_stats:
    break 
  
  best_pair = max(pair_stats, key=pair_stats.get)
  bpe_codes[best_pair] = i
  print('vocabulary: ', vocab)
  print('best pair:', best_pair)
  vocab = merge_vocab(best_pair, vocab)

print("\n vocab: ", vocab)
print('byte pair encoding: ', bpe_codes)

iter:  0
##################
word: l o w </w>, freq: 5
symbols:  ['l', 'o', 'w', '</w>']
pair:  ('l', 'o')
current_freq:  0
pairs[pair]:  5
pair:  ('o', 'w')
current_freq:  0
pairs[pair]:  5
pair:  ('w', '</w>')
current_freq:  0
pairs[pair]:  5
################## 

##################
word: l o w e r </w>, freq: 2
symbols:  ['l', 'o', 'w', 'e', 'r', '</w>']
pair:  ('l', 'o')
current_freq:  5
pairs[pair]:  7
pair:  ('o', 'w')
current_freq:  5
pairs[pair]:  7
pair:  ('w', 'e')
current_freq:  0
pairs[pair]:  2
pair:  ('e', 'r')
current_freq:  0
pairs[pair]:  2
pair:  ('r', '</w>')
current_freq:  0
pairs[pair]:  2
################## 

##################
word: n e w e s t </w>, freq: 6
symbols:  ['n', 'e', 'w', 'e', 's', 't', '</w>']
pair:  ('n', 'e')
current_freq:  0
pairs[pair]:  6
pair:  ('e', 'w')
current_freq:  0
pairs[pair]:  6
pair:  ('w', 'e')
current_freq:  2
pairs[pair]:  8
pair:  ('e', 's')
current_freq:  0
pairs[pair]:  6
pair:  ('s', 't')
current_freq:  0
pairs[pair]:  6
pair:  (

# Encode

In [ ]:
original_word = 'lowest'
word = list(original_word)
word.append('</w>')
word

['l', 'o', 'w', 'e', 's', 't', '</w>']

In [ ]:
# lấy các cặp ký tự trong word input
def get_pairs(word: List[str]) -> Set[Tuple[str, str]]:
  pairs = set()
  prev_char = word[0]
  for char in word[1:]:
    pairs.add((prev_char, char))
    prev_char = char
  
  return pairs

In [ ]:
pairs = get_pairs(word)
pairs

{('e', 's'), ('l', 'o'), ('o', 'w'), ('s', 't'), ('t', '</w>'), ('w', 'e')}

In [ ]:
bpe_codes_pairs = [(pair, bpe_codes[pair]) for pair in pairs if pair in bpe_codes]
print("bpe_codes_pairs: ", bpe_codes_pairs)
pair_to_merge = min(bpe_codes_pairs, key=itemgetter(1))[0]
print("pair_to_merge: ", pair_to_merge)

bpe_codes_pairs:  [(('l', 'o'), 3), (('e', 's'), 0)]
pair_to_merge:  ('e', 's')


In [ ]:
def create_new_word(word: List[str], pair_to_merge: Tuple[str, str]) -> List[str]:
  first_char, second_char = pair_to_merge
  new_word = []
  i = 0
  while i < len(word):
    try:
      j = word.index(first_char, i)
      new_word.extend(word[i:j])
      i = j
    except:
      new_word.extend(word[i:])
      break 
    
    if i < len(word)-1 and word[i+1] == second_char:
      new_word.append(first_char + second_char)
      i += 2
    else:
      new_word.append(first)
      i += 1

  return new_word

In [ ]:
new_word = create_new_word(word, pair_to_merge)
new_word

['l', 'o', 'w', 'es', 't', '</w>']

Phần code trên là quá trình thực hiện cho việc encode 1 từ qua 1 iter. Bây giờ thực hiện với nhiều iter.


In [ ]:
def encode(original_word: str, bpe_codes: Dict[Tuple[str, str], int], len_word_split=1) -> List[str]:
  if len(original_word) == len_word_split: # set kích thước tối thiểu của một từ cần phân tách.
    return original_word
  
  word = list(original_word)
  word.append('</w>')

  while True:
    pairs = get_pairs(word)
    bpe_codes_pairs = [(pair, bpe_codes[pair]) for pair in pairs if pair in bpe_codes]
    if not bpe_codes_pairs:
        break

    pair_to_merge = min(bpe_codes_pairs, key=itemgetter(1))[0]
    word = create_new_word(word, pair_to_merge)
  
  return word

In [ ]:
original_word = 'lowest'
encode(original_word, bpe_codes)

['low', 'est</w>']